In [15]:
import ipywidgets as widgets
import numpy as np
from abtem import Probe
from abtem.visualize.interactive.artists import ImageArtist
from abtem.visualize.interactive.utils import throttle
from abtem.visualize.interactive.canvas import Canvas
from abtem.visualize.utils import domain_coloring


from IPython.core.display import display, HTML
display(HTML("<style>.container { width:60% !important; }</style>"))

In [16]:
canvas1 = Canvas()
canvas2 = Canvas()

artist1 = ImageArtist(rgb=True)
canvas1.artists = {'image': artist1}

artist2 = ImageArtist(rgb=True)
canvas2.artists = {'image': artist2}

probe = Probe(energy=80e3, semiangle_cutoff=30, defocus=0,
              Cs=0e4, astigmatism=0, gpts=128, extent=20, rolloff=2)

aperture_slider = widgets.FloatSlider(description='Aperture', min=1, max=100, value=30, step=1)
energy_slider = widgets.FloatSlider(description='Energy', min=50e3, max=200e3, value=100e3, step=10e3)
defocus_slider = widgets.FloatSlider(description='Defocus', min=-200, max=200, value=0, step=10)
cs_slider = widgets.FloatSlider(description='Cs', min=-5e5, max=5e5, value=0, step=1e4)

color_dropdown = widgets.Dropdown(description='Colormap', options=['Intensity', 'Complex', 'Amplitude', 'Phase', ],
                                  value='Intensity')


def update(*args):
    probe.ctf.semiangle_cutoff = aperture_slider.value
    probe.ctf.defocus = defocus_slider.value
    probe.ctf.energy = energy_slider.value
    probe.ctf.Cs = cs_slider.value

    ctf_array = probe.ctf.evaluate_on_grid(probe.gpts[0], probe.extent[0] * 3 / 2)
    probe_array = np.fft.fftshift(np.fft.fft2(ctf_array))
    ctf_array = np.fft.fftshift(ctf_array)

    with artist1._mark.hold_sync():
        if color_dropdown.value == 'Intensity':
            artist1.image = np.tile((np.abs(ctf_array) ** 2)[..., None], (1, 1, 3))
            intensity = np.abs(probe_array) ** 2
            artist2.image = np.tile((intensity / intensity.max())[..., None], (1, 1, 3))
        elif color_dropdown.value == 'Complex':
            artist1.image = domain_coloring(ctf_array)
            artist2.image = domain_coloring(probe_array)
        elif color_dropdown.value == 'Amplitude':
            artist1.image = np.tile((np.abs(ctf_array))[..., None], (1, 1, 3))
            intensity = np.abs(probe_array)
            artist2.image = np.tile((intensity / intensity.max())[..., None], (1, 1, 3))
        elif color_dropdown.value == 'Phase':
            artist1.image = domain_coloring(np.exp(1.j * np.angle(ctf_array)), pure_phase=True)
            artist2.image = domain_coloring(np.exp(1.j * np.angle(probe_array)), pure_phase=True)

        artist1.extent = [[-probe.cutoff_scattering_angles[0], probe.cutoff_scattering_angles[0]]] * 2


def reset_button_callback(*args):
    with artist1._mark.hold_sync(), artist2._mark.hold_sync():
        aperture_slider.value = 30
        defocus_slider.value = 0
        energy_slider.value = 100e3
        cs_slider.value = 0

update()

artist2.extent = [[0, probe.extent[0]], [0, probe.extent[1]]]

update = throttle(.1)(update)
aperture_slider.observe(update, 'value')
energy_slider.observe(update, 'value')
defocus_slider.observe(update, 'value')
cs_slider.observe(update, 'value')
color_dropdown.observe(update, 'value')

canvas1.x_label = 'Scattering angle x [mrad.]'
canvas1.y_label = 'Scattering angle y [mrad.]'
canvas1.title = 'Fourier space probe'

canvas2.x_label = 'x [Å]'
canvas2.y_label = 'y [Å]'
canvas2.title = 'Real space probe'

whitespace = widgets.HBox([])
whitespace.layout.height = '30px'

reset_button = widgets.Button(description='Reset')

reset_button.on_click(reset_button_callback)

widgets.HBox([canvas1.widget, canvas2.widget,
              widgets.VBox([whitespace,
                            color_dropdown,
                            aperture_slider,
                            energy_slider,
                            defocus_slider,
                            cs_slider,
                            reset_button])])